# Advent of Code 2020

Programming excercises from [Advent of Code](https://adventofcode.com), each day has two programming exercises. Most of the good ideas in here are probably borrowed from [Peter Norvig's solutions](https://github.com/norvig/pytudes/blob/master/ipynb/Advent-2020.ipynb).

## Day 0

Imports & Basic structure

In [4]:
#from fastcore import *

from typing      import Set, Dict, List, Tuple
from itertools   import combinations
from collections import Counter, namedtuple

import math
import re
import sys
import operator

In [5]:
def get_data(day: int, parser=int, sep='\n') -> list:
    """Returns a list of elements in the day's input file.
    Separated by `sep`, `parser` applied to each element."""
    with open(f'AoC2020/input{day}.txt') as file:
        splits = file.read().rstrip().split(sep)
        return [parser(x) for x in splits]

def do(day: int, *answers):
    global_scope = globals()
    results = []
    for part in (1, 2): 
        function_name = f'day{day}_{part}'
        if function_name in global_scope:
            result = global_scope[function_name](global_scope[f'input{day}'])
            if answers:
                assert result == answers[part-1]
            results.append(result)
    return results

Utility functions

In [6]:
def first(iterable):
    return next(iter(iterable))

def prod(*numbers):
    result = 1
    for n in numbers:
        result *= n
    return result

def get_numbers(text: str) -> Tuple:
    findings = re.findall('\d+', text)
    return tuple([int(x) for x in findings])

def quantify(iterable, condition=bool) -> int:
    return sum([1 for item in iterable if condition(item)])

Char = str # Type that indicates a single letter

In [7]:
assert prod(2, 3, 4) == 24, "2*3*4 should be 24"
assert get_numbers("there are 3 numbers in here. 2 are small one is big: 42") == (3, 2, 42)
assert quantify([True, False, True]) == 2

## Day 1

Find the two entries that sum to 2020; what do you get if you multiply them together?

Naive / brute force solution

In [8]:
numbers = get_data(1)
for n1 in numbers:
    for n2 in numbers:
        if n1 + n2 == 2020:
            result = n1*n2
            break
print(result)

793524


More elegant & efficient solution (@norvig)

In [9]:
input1: Set[int] = set(get_data(1))

def day1_1(nums):
    return first(
        x * y
        for x in nums
        for y in nums & {2020 - x}
        if x != y
    )
do(1)

[793524]

In your expense report, what is the product of the **three entries** that sum to 2020?

In [10]:
def day1_2(nums):
    return first(
        x * y * z
        for x, y in combinations(nums, 2)
        for z in nums & {2020 - x - y}
        if x != y != z
    )

do(1, 793524, 61515678)

[793524, 61515678]

## Day 2

How many passwords are valid according to their policies?

Policy: Password is valid when a letter is between `a` and `b` times in the password.

In [11]:
Policy = Tuple[int, int, Char, str]

def pwd_parser(line: str) -> Policy:
    "Turns '2-4 r: prrmspx' into (2, 4, 'r', 'prrmspx')"
    low, high, letter, pwd = re.findall(r'[^-:\s]+', line)
    return (int(low), int(high), letter, pwd)

def is_valid_pwd(policy: Policy) -> bool:
    low, high, letter, pwd = policy
    return low <= pwd.count(letter) <= high

In [12]:
input2 = get_data(2, parser=pwd_parser)

def day2_1(policies: List[Policy]):
    return quantify(policies, is_valid_pwd)

do(2)

[467]

New policy: the `password` must have `letter` at positions `a` or `b`, but not at both.

In [13]:
def is_valid_pwd_2(policy: Policy) -> bool:
    a, b, letter, pwd = policy
    return (pwd[a-1] == letter) ^ (pwd[b-1] == letter)

def day2_2(policies: List[Policy]):
    return quantify(policies, is_valid_pwd_2)

do(2, 467, 441)

[467, 441]

## Day 3

We have a picture that looks something like this:

```
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
```

The lines repeat to the right.

We start in the top left and traverse the picture in a diagonal line (slope: 1 down, 3 right). How many trees `#` do we count?

In [14]:
Picture = List[str]
input3: Picture = get_data(3, parser=str)

def day3_1(picture, dx=3, dy=1):
    traversed = [(line[(dx*y) % len(line)] == '#')
                 for y, line in enumerate(picture[::dy])]
    return quantify(traversed)

In [15]:
do(3)

[244]

What if we multiplied the encountered trees for several slopes?

In [16]:
slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
def day3_2(picture):
    return prod(*[day3_1(picture, dx, dy) for dx, dy in slopes])

do(3, 244, 9406609920)

[244, 9406609920]

## Day 4